<a href="https://colab.research.google.com/github/vnicula/ai-research-code/blob/master/algorithms/multivariate_regression_sgd_hidden_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [16]:
df = pd.read_csv('https://raw.githubusercontent.com/satishgunjal/datasets/master/multivariate_housing_prices_in_portlans_oregon.csv')
print(df.size)
df.head()

141


,size(in square feet),number of bedrooms,price
0,2104,3,399900
1,1600,3,329900
2,2400,3,369000
3,1416,2,232000
4,3000,4,539900


In [52]:
X_train = df.iloc[:, 0:2].values
# print(np.std(X_train, axis=0, keepdims=True)[:3])
Y_train = np.expand_dims(df.iloc[:, 2].values, 1)
X_train = (X_train - X_train.mean(axis=0, keepdims=True)) / np.std(X_train, axis=0, keepdims=True)
print(X_train[:3], Y_train[:3])


[[ 0.13141542 -0.22609337]
 [-0.5096407  -0.22609337]
 [ 0.5079087  -0.22609337]] [[399900]
 [329900]
 [369000]]


In [63]:
def init_weights():
  W1 = np.random.normal(loc=0, scale=0.1, size=[10, 2])
  B1 = np.random.normal(loc=0, scale=0.1, size=[10, 1])
  W2 = np.random.normal(loc=0, scale=0.1, size=[1, 10])
  B2 = np.random.normal(loc=0, scale=0.1, size=[1, 1])
  return W1, B1, W2, B2

# print(init_weights())

In [19]:
def ReLU(z):
  return np.maximum(z, 0)

def loss(Y, Yh):
  return 0.5*(Y-Yh)**2

def forward_pass(X, W1, B1, W2, B2):
  Z1 = W1.dot(X) + B1 # (3, 1)
  A1 = ReLU(Z1)       # (3, 1)
  O = W2.dot(A1) + B2 # (1, 1)
  return Z1, A1, O


In [20]:
def backward_pass(X, Y, Z1, A1, O, W1, B1, W2, B2):
  # last layer
  dL_O = O - Y # (1, 1)
  dO_W2 = A1   # (3, 1)
  gW2 = dL_O.dot(A1.T)
  gB2 = dL_O

  # hidden layer
  # local gradient dO_Z1 = dO_A1 * dA1_Z1
  dO_A1 = W2   # (1, 3)
  dA1_Z1 = (Z1 > 0) * 1.0  # (3, 1)
  delta2 = dL_O.dot(W2 * dA1_Z1.T) # (1,3)
  gW1 = delta2.T.dot(X.T)
  gB1 = delta2.T

  return gW1, gB1, gW2, gB2

In [42]:
def train_step(X, Y, W1, B1, W2, B2, steps, lr):
  num_samples = X.T.shape[1]
  print(f"num_samples={num_samples}")

  for i in range(steps):
    losses = []
    for j in range(num_samples):
      sample = X[j].reshape([2,1])
      y = Y[j] / 10000.0
      Z1, A1, O = forward_pass(sample, W1, B1, W2, B2)
      # print(j, O)
      losses.append(loss(y, O))
      gW1, gB1, gW2, gB2 = backward_pass(sample, y, Z1, A1, O, W1, B1, W2, B2)
      # print(gW1)
      W1 -= lr*gW1
      B1 -= lr*gB1
      W2 -= lr*gW2
      B2 -= lr*gB2
    
    print("Avg loss: ", np.mean(losses))



In [64]:
W1, B1, W2, B2 = init_weights()
train_step(X_train, Y_train, W1, B1, W2, B2, 150, 0.0005)

num_samples=47
Avg loss:  643.4774836277134
Avg loss:  611.7738595982091
Avg loss:  563.7511883410837
Avg loss:  453.72854390398254
Avg loss:  240.69330610909708
Avg loss:  70.74396266983241
Avg loss:  28.458390164164722
Avg loss:  23.306099648854662
Avg loss:  22.465983044786128
Avg loss:  22.194654270681358
Avg loss:  22.066895994384016
Avg loss:  21.99811598912555
Avg loss:  21.959329080975643
Avg loss:  21.936002616743263
Avg loss:  21.91838892632662
Avg loss:  21.901477311816322
Avg loss:  21.886051382542025
Avg loss:  21.872013689053855
Avg loss:  21.859054655375065
Avg loss:  21.846476441813685
Avg loss:  21.835628209257827
Avg loss:  21.82510011357441
Avg loss:  21.818304603217996
Avg loss:  21.812746986936716
Avg loss:  21.807727825021058
Avg loss:  21.802962246659
Avg loss:  21.79836355122038
Avg loss:  21.793896477756878
Avg loss:  21.78365375300104
Avg loss:  21.77440572235137
Avg loss:  21.764617459527475
Avg loss:  21.754831972939176
Avg loss:  21.745200480157486
Avg loss